In [1]:
import pandas as pd
import numpy as np
import os

# 1. Загружаем данные из CSV файла
file_path = r"C:\Users\elena.mordvinova\PYT\L1\1_Airline Delay-задержка рейса .csv"

# Проверяем наличие файла
if os.path.exists(file_path):
    try:
        # Читаем CSV файл
        df = pd.read_csv(file_path, encoding='utf-8')  # Попробуй 'ISO-8859-1', если возникнут проблемы
        print("Начальные данные:")
        print(df.head(), "\n")  # Выводим первые строки DataFrame

        # 2. Делим DataFrame на две части и объединяем
        df1 = df.iloc[:len(df)//2]  # первая половина
        df2 = df.iloc[len(df)//2:]  # вторая половина
        df_concat = pd.concat([df1, df2], ignore_index=True)
        print("Объединенный DataFrame:")
        print(df_concat.head(), "\n")  # Выводим первые строки объединенного DataFrame

        # 3. Проверяем и удаляем дубликаты
        if df_concat.duplicated().any():
            df_concat = df_concat.drop_duplicates()
            print("Дубликаты удалены.\n")
        else:
            print("Дубликатов не найдено.\n")

        # 4. Проверяем на пропуски
        if df_concat.isnull().any().any():
            print("Пропуски найдены.\n")
        else:
            # Добавляем случайный пропуск
            index_to_add_nan = np.random.choice(df_concat.index, 1)[0]
            df_concat.loc[index_to_add_nan, df_concat.columns[0]] = np.nan
            print("Пропуски добавлены.\n")

        # 5. Заполняем пропуски средними значениями
        df_concat.fillna(value=df_concat.mean(numeric_only=True), inplace=True)
        print("Пропуски заполнены средними значениями:")
        print(df_concat.head(), "\n")

        # 6. Проверяем на выбросы (IQR)
        Q1 = df_concat.quantile(0.25)
        Q3 = df_concat.quantile(0.75)
        IQR = Q3 - Q1
        outliers_condition = ((df_concat < (Q1 - 1.5 * IQR)) | (df_concat > (Q3 + 1.5 * IQR))).any(axis=1)

        if outliers_condition.any():
            df_concat = df_concat[~outliers_condition]
            print("Выбросы удалены.\n")
        else:
            print("Выбросов не найдено.\n")

        # 7. Нормализуем данные (Min-Max Normalization)
        df_normalized = (df_concat - df_concat.min()) / (df_concat.max() - df_concat.min())
        print("Нормализованные данные:")
        print(df_normalized.head(), "\n")

    except pd.errors.EmptyDataError:
        print("Файл пуст.")
    except pd.errors.ParserError:
        print("Ошибка разбора файла.")
    except Exception as e:
        print(f"Ошибка: {e}")
else:
    print("Файл не найден. Проверь путь к файлу:")
    print(file_path)

Начальные данные:
   year  month carrier       carrier_name airport  \
0  2020     12      9E  Endeavor Air Inc.     ABE   
1  2020     12      9E  Endeavor Air Inc.     ABY   
2  2020     12      9E  Endeavor Air Inc.     AEX   
3  2020     12      9E  Endeavor Air Inc.     AGS   
4  2020     12      9E  Endeavor Air Inc.     ALB   

                                        airport_name  arr_flights  arr_del15  \
0  Allentown/Bethlehem/Easton, PA: Lehigh Valley ...         44.0        3.0   
1             Albany, GA: Southwest Georgia Regional         90.0        1.0   
2           Alexandria, LA: Alexandria International         88.0        8.0   
3        Augusta, GA: Augusta Regional at Bush Field        184.0        9.0   
4                   Albany, NY: Albany International         76.0       11.0   

   carrier_ct  weather_ct  ...  security_ct  late_aircraft_ct  arr_cancelled  \
0        1.63         0.0  ...          0.0              1.25            0.0   
1        0.96         

C:\Users\elena.mordvinova\AppData\Local\Temp\ipykernel_9560\1049104448.py:45: FutureWarning: The default value of numeric_only in DataFrame.quantile is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  Q1 = df_concat.quantile(0.25)
C:\Users\elena.mordvinova\AppData\Local\Temp\ipykernel_9560\1049104448.py:46: FutureWarning: The default value of numeric_only in DataFrame.quantile is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  Q3 = df_concat.quantile(0.75)
C:\Users\elena.mordvinova\AppData\Local\Temp\ipykernel_9560\1049104448.py:48: FutureWarning: Automatic reindexing on DataFrame vs Series comparisons is deprecated and will raise ValueError in a future version. Do `left, right = left.align(right, axis=1, copy=False)` before e.g. `left == right`
  outliers_condition = ((df_concat < (Q1 - 